In [2]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MaxAbsScaler
from sklearn.cluster import AgglomerativeClustering, DBSCAN, KMeans
import gower
from scipy.cluster.hierarchy import dendrogram
from sklearn.decomposition import PCA


__DATENVORBEREITUNG__

In [3]:
# load data
artikel = pd.read_excel(r"C:\Users\IM-KI\Documents\PrEvelop_Daten_mkPlast\PrEvelOp_mkPlast_01_Esb.xlsx")
arbeitsplanzeilen = pd.read_excel(r"C:\Users\IM-KI\Documents\PrEvelop_Daten_mkPlast\PrEvelOp_mkPlast_01_Esb_Arbeitszeilen.xlsx")

In [4]:
# set identifier
artikel.rename(columns={'Nr.': 'index'}, inplace=True)
artikel.set_index('index', inplace=True)
display(artikel.head(5))
arbeitsplanzeilen.rename(columns={'Arbeitsplannr.': 'index'}, inplace=True)
arbeitsplanzeilen.set_index('index', inplace=True)
display(arbeitsplanzeilen.head(5))

,Beschreibung,Beschreibung 2,Mengeneinheit,Rohstoff,Artikelgruppe,Produktgruppe,Foliencode,Farbcode,Arbeitsplannr.,Breite(mm),...,Rollendurchmesser(mm),Rollenlänge(m),Stk pro Rolle,Randbeschnitt(mm),Bündelung(stk),Gewicht / 1000m,Echt Gewicht / 1000m,Gewicht / 1000skt,Echt Gewicht / 1000skt,MFI
index,,,,,,,,,,,,,,,,,,,,,
100012,MDPE Seitenfaltenbeutel,-Ascalon Umbeutel-,STK,MDPE,KONFEKTION,SFB,MT-LOS-,TRANSPARENT,100012,180,...,0,0.0,0,0,0,22.5120,22.463061,10.580640,10.557639,0.0
100013,HDPE Seitenfaltenbeutel,NaN,STK,HDPE,KONFEKTION,SFB,BL-,BLAU DUNKEL-2,100013,580,...,0,0.0,0,0,50,19.4616,0.000000,12.844656,0.000000,0.0
100014,LDPE Flachfolienzuschnitte,NaN,STK,LDPE,KONFEKTION,ZUS,ST-,TRANSPARENT,100014,600,...,0,0.0,0,0,500,11.0880,0.000000,7.761600,0.000000,0.0
100015,HDPE Seitenfaltenbeutel,NaN,STK,HDPE,KONFEKTION,SFB,LME-ST-,BLAU HELL-2,100015,400,...,0,0.0,0,0,500,13.3560,13.340224,4.674600,4.669078,0.0
100017,MDPE Flachbeutel,NaN,STK,MDPE,KONFEKTION,FB,LME-LOS-,ROT 5500-3,100017,395,...,0,0.0,0,0,0,33.3459,32.936424,24.675966,24.372954,0.0


,Versionscode,Arbeitsgangnr.,Nächste Arbeitsgangnr.,Vorherige Arbeitsgangnr.,Art,Nr.,Arbeitsplatzgruppennr.,Abteilungscode,Beschreibung,Rüstzeit,...,Reihenfolge (Rückwärts),Kum. feste Ausschussmenge,Kum. Ausschussfaktor %,Takt,kalk. Durchsatz,kalk. Bearbeitungszeit,Durchsatz,Nutzen (Bahnigkeit),Nutzen (Lagigkeit),Abzugsgeschwindigkeit
index,,,,,,,,,,,,,,,,,,,,,
100012,NaN,100,NaN,NaN,Arbeitsplatzgruppe,21.0,21.0,KONFEKTION,BSZ II,0.5,...,1,0,0.0,0.0,0.0,0.0,25000.0,3,1,0.0
100013,NaN,100,NaN,NaN,Arbeitsplatzgruppe,21.0,21.0,KONFEKTION,BSZ II,0.5,...,1,0,0.0,0.0,0.0,0.0,0.0,2,1,0.0
100014,NaN,100,NaN,NaN,Arbeitsplatzgruppe,20.0,20.0,KONFEKTION,BSZ I,0.5,...,1,0,0.0,0.0,0.0,0.0,0.0,2,4,0.0
100015,NaN,100,NaN,NaN,Arbeitsplatzgruppe,21.0,21.0,KONFEKTION,BSZ II,0.5,...,1,0,0.0,140.0,0.0,0.0,16800.0,2,1,0.0
100016,NaN,100,NaN,NaN,Arbeitsplatzgruppe,3.0,3.0,EXTRUSION,EX3,0.5,...,1,0,0.0,0.0,0.0,0.0,0.0,1,1,0.0


In [5]:
# check if identifier is unique
print(artikel.index.is_unique)
print(arbeitsplanzeilen.index.is_unique)
# count values in Arbeitsplannr.
artikel['Arbeitsplannr.'].value_counts()

True
False


Arbeitsplannr.
300322    82
102235    63
107616    20
100195    11
103802    10
          ..
105879     1
105880     1
105881     1
105882     1
306171     1
Name: count, Length: 13548, dtype: int64

In [6]:
# drop row in arbeitsplatzeilen with duplicate index
arbeitsplanzeilen = arbeitsplanzeilen[~arbeitsplanzeilen.index.duplicated(keep='first')]


In [7]:
# count nan values of Arbeitsplannr. in artikel
artikel['Arbeitsplannr.'].isna().sum()

1097

In [8]:
# drop rows with nan values in Arbeitsplannr.
artikel = artikel.dropna(subset=['Arbeitsplannr.'])

In [9]:
artikel.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15165 entries, 100012 to 700002
Data columns (total 26 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Beschreibung            15165 non-null  object 
 1   Beschreibung 2          1988 non-null   object 
 2   Mengeneinheit           15165 non-null  object 
 3   Rohstoff                15165 non-null  object 
 4   Artikelgruppe           15164 non-null  object 
 5   Produktgruppe           15164 non-null  object 
 6   Foliencode              14445 non-null  object 
 7   Farbcode                15161 non-null  object 
 8   Arbeitsplannr.          15165 non-null  object 
 9   Breite(mm)              15165 non-null  int64  
 10  Seitenfalte(mm)         15165 non-null  int64  
 11  Länge(mm)               15165 non-null  int64  
 12  Stärke(mm)              15165 non-null  float64
 13  Echte Stärke(mm)        15165 non-null  float64
 14  Dichte(g/cm³)           15165 non-nul

In [10]:
# drop column 'Beschreibung 2'
artikel.drop(columns=['Beschreibung 2'], inplace=True)

In [11]:
# count number of rows with zero values for all columns in artikel
for column in artikel.columns:
    print(str(column)+':', (artikel[column] == 0).sum())

Beschreibung: 0
Mengeneinheit: 0
Rohstoff: 0
Artikelgruppe: 0
Produktgruppe: 0
Foliencode: 0
Farbcode: 0
Arbeitsplannr.: 0
Breite(mm): 2
Seitenfalte(mm): 8527
Länge(mm): 8719
Stärke(mm): 2
Echte Stärke(mm): 976
Dichte(g/cm³): 8
Rollengewicht (kg): 15085
Rollendurchmesser(mm): 14250
Rollenlänge(m): 13046
Stk pro Rolle: 15160
Randbeschnitt(mm): 14226
Bündelung(stk): 12364
Gewicht / 1000m: 6
Echt Gewicht / 1000m: 1675
Gewicht / 1000skt: 8717
Echt Gewicht / 1000skt: 9432
MFI: 15165


In [12]:
# drop columns 
columns_keep = ['Arbeitsplatzgruppennr.','Abteilungscode','Beschreibung','Rüstzeit','Bearbeitungszeit','Takt','kalk. Durchsatz','kalk. Bearbeitungszeit','Durchsatz','Nutzen (Bahnigkeit)','Nutzen (Lagigkeit)']
arbeitsplanzeilen = arbeitsplanzeilen[columns_keep]

In [13]:
arbeitsplanzeilen.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13926 entries, 100012 to 77
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Arbeitsplatzgruppennr.  13923 non-null  float64
 1   Abteilungscode          13923 non-null  object 
 2   Beschreibung            13922 non-null  object 
 3   Rüstzeit                13926 non-null  float64
 4   Bearbeitungszeit        13926 non-null  float64
 5   Takt                    13926 non-null  float64
 6   kalk. Durchsatz         13926 non-null  float64
 7   kalk. Bearbeitungszeit  13926 non-null  float64
 8   Durchsatz               13926 non-null  float64
 9   Nutzen (Bahnigkeit)     13926 non-null  int64  
 10  Nutzen (Lagigkeit)      13926 non-null  int64  
dtypes: float64(7), int64(2), object(2)
memory usage: 1.3+ MB


In [14]:
#count number of rows with zero values for all columns in arbeitsplanzeilen
for column in arbeitsplanzeilen.columns:
    print(str(column)+':', (arbeitsplanzeilen[column] == 0).sum())

Arbeitsplatzgruppennr.: 0
Abteilungscode: 0
Beschreibung: 0
Rüstzeit: 76
Bearbeitungszeit: 239
Takt: 10356
kalk. Durchsatz: 12808
kalk. Bearbeitungszeit: 12794
Durchsatz: 3571
Nutzen (Bahnigkeit): 0
Nutzen (Lagigkeit): 0


In [15]:
list_a = artikel['Arbeitsplannr.'].unique().tolist()
list_b = arbeitsplanzeilen.index.unique().tolist()
print(len(list_a))
print(len(list_b))
# change type of list_b to string
list_b = [str(i) for i in list_b]
# count number of values in list_a that are not in list_b
print(len(set(list_a) - set(list_b)))
# show values in list_a that are not in list_b
print(set(list_a) - set(list_b))

13548
13926
7
{'108181', '304936', '108177', '110121', '108182', '108183', '300645'}


In [16]:
drop_list = list(set(list_a) - set(list_b))
# drop rows in artikel with Arbeitsplannr. in drop_list
artikel = artikel[~artikel['Arbeitsplannr.'].isin(drop_list)]

In [17]:
# change dtype of index in arbeitsplanzeilen to string
arbeitsplanzeilen.index = arbeitsplanzeilen.index.astype(str)

In [18]:
# merge artikel and arbeitsplanzeilen
mkplast = pd.merge(artikel, arbeitsplanzeilen, left_on='Arbeitsplannr.', right_index=True, how='left')

In [19]:
mkplast

,Beschreibung_x,Mengeneinheit,Rohstoff,Artikelgruppe,Produktgruppe,Foliencode,Farbcode,Arbeitsplannr.,Breite(mm),Seitenfalte(mm),...,Abteilungscode,Beschreibung_y,Rüstzeit,Bearbeitungszeit,Takt,kalk. Durchsatz,kalk. Bearbeitungszeit,Durchsatz,Nutzen (Bahnigkeit),Nutzen (Lagigkeit)
index,,,,,,,,,,,,,,,,,,,,,
100012,MDPE Seitenfaltenbeutel,STK,MDPE,KONFEKTION,SFB,MT-LOS-,TRANSPARENT,100012,180,120,...,KONFEKTION,BSZ II,0.5,0.00004,0.0,0.0,0.0,25000.00,3,1
100013,HDPE Seitenfaltenbeutel,STK,HDPE,KONFEKTION,SFB,BL-,BLAU DUNKEL-2,100013,580,440,...,KONFEKTION,BSZ II,0.5,0.00000,0.0,0.0,0.0,0.00,2,1
100014,LDPE Flachfolienzuschnitte,STK,LDPE,KONFEKTION,ZUS,ST-,TRANSPARENT,100014,600,0,...,KONFEKTION,BSZ I,0.5,0.00000,0.0,0.0,0.0,0.00,2,4
100015,HDPE Seitenfaltenbeutel,STK,HDPE,KONFEKTION,SFB,LME-ST-,BLAU HELL-2,100015,400,300,...,KONFEKTION,BSZ II,0.5,0.00006,140.0,0.0,0.0,16800.00,2,1
100017,MDPE Flachbeutel,STK,MDPE,KONFEKTION,FB,LME-LOS-,ROT 5500-3,100017,395,0,...,KONFEKTION,BF 1600 I,0.5,0.00005,0.0,0.0,0.0,20000.00,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306168,MDPE Seitenfaltenschlauch,KG,MDPE,EXTRUSION,SFSF,LME-,BLAU HELL-1,306168,570,440,...,EXTRUSION,EX7,0.5,0.00556,180.0,0.0,0.0,180.00,1,1
306169,MDPE Seitenfaltenschlauch,KG,MDPE,EXTRUSION,SFSF,LME-,BLAU HELL-1,306169,400,340,...,EXTRUSION,EX8,0.5,0.00667,150.0,0.0,0.0,150.00,1,1
306170,HDPE Seitenfaltenschlauch,KG,HDPE,EXTRUSION,SFSF,RO-,TRANSPARENT,306170,570,460,...,EXTRUSION,EX1,0.5,0.01111,90.0,0.0,0.0,90.00,1,1


In [20]:
mkplast.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15158 entries, 100012 to 700002
Data columns (total 36 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Beschreibung_x          15158 non-null  object 
 1   Mengeneinheit           15158 non-null  object 
 2   Rohstoff                15158 non-null  object 
 3   Artikelgruppe           15157 non-null  object 
 4   Produktgruppe           15157 non-null  object 
 5   Foliencode              14439 non-null  object 
 6   Farbcode                15155 non-null  object 
 7   Arbeitsplannr.          15158 non-null  object 
 8   Breite(mm)              15158 non-null  int64  
 9   Seitenfalte(mm)         15158 non-null  int64  
 10  Länge(mm)               15158 non-null  int64  
 11  Stärke(mm)              15158 non-null  float64
 12  Echte Stärke(mm)        15158 non-null  float64
 13  Dichte(g/cm³)           15158 non-null  float64
 14  Rollengewicht (kg)      15158 non-nul

__CLUSTERING__

In [21]:
# calculate distance matrix with gower distance
distance_matrix = gower.gower_matrix(mkplast)

In [25]:
# cluster with KMeans
# iterate over different number of clusters and calculate the sum of the variance of the clusters
cluster_variance = []
for k in range(1,20):
    clustering = KMeans(n_clusters=k).fit(distance_matrix)
    cluster_variance_sum = []
    for i in range(k):
        cluster_indices = []
        for index, cluster in enumerate(list(clustering.labels_)):
            if cluster == i:
                cluster_indices.append(index)
        cluster_len = len(cluster_indices)
        cluster_df = mkplast.iloc[cluster_indices]
        cluster_df.loc['metoid'] = cluster_df.mean()
        cluster_variance = sum(gower.gower_matrix(cluster_df)[-1])
        cluster_variance_sum.append(cluster_variance)
    cluster_variance.append(sum(cluster_variance_sum))

C:\Users\IM-KI\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


TypeError: can only concatenate str (not "int") to str

cluster
 1    9082
 0    5925
-1     120
 4      13
 5       7
 2       6
 3       5
Name: count, dtype: int64